
<a href="http://landlab.github.io"><img style="float: left; width: 300px;" src="https://landlab.csdms.io/_static/landlab_logo.png"></a>

# 2D Surface Water Flow component

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>


## Overview

This notebook demonstrate the usage of the `river flow dynamics` Landlab component. The component runs a semi-implicit, semi-Lagrangian finite-volume approximation to the depth-averaged 2D shallow-water equations of Casulli and Cheng (1992) and related work.

### Theory

The depth-averaged 2D shallow-water equations are the simplification of the Navier-Stokes equations, which correspond to the balance of momentum and mass in the fluid. It is possible to simplify these equations by assuming a well-mixed water column and a small water depth to width ratio, where a vertical integration results in depth-averaged equations. These require boundary conditions at the top and bottom of the water column, which are provided by the wind stress and the Manning-Chezy formula, respectively:

$$
\frac{\partial U}{\partial t}
+ U\frac{\partial U}{\partial x} + V\frac{\partial U}{\partial y}
= 
- g\frac{\partial \eta}{\partial x}
+ \epsilon\left(\frac{\partial^2 U}{\partial x^2} + \frac{\partial^2 U}{\partial y^2}\right)
+ \frac{\gamma_T(U_a - U)}{H} - g\frac{\sqrt{U^2 + V^2}}{Cz^2}U + \mathbf{f}V
$$

$$
\frac{\partial V}{\partial t}
+ U\frac{\partial V}{\partial x} + V\frac{\partial V}{\partial y}
= 
- g\frac{\partial \eta}{\partial y}
+ \epsilon\left(\frac{\partial^2 V}{\partial x^2} + \frac{\partial^2 V}{\partial y^2}\right)
+ \frac{\gamma_T(V_a - V)}{H} - g\frac{\sqrt{U^2 + V^2}}{Cz^2}V + \mathbf{f}U
$$

$$
\frac{\partial \eta}{\partial t}
+ \frac{\partial (HU)}{\partial x} + \frac{\partial (HV)}{\partial y}
= 0
$$

where $U$ is the water velocity in the $x$-direction, $V$ is the water velocity in the $y$-direction, $H$ is the water depth, $\eta$ is the water surface elevation, $Cz$ is the Chezy friction coefficient, and $t$ is time. For the constants $g$ is the gravity acceleration, $\epsilon$ is the horizontal eddy viscosity, $\mathbf{f}$ is the Coriolis parameter, $\gamma_T$ is the wind stress coefficient, and $U_a$ and $V_a$ are the prescribed wind velocities.

### Numerical representation

A semi-implicit, semi-Lagrangian, finite volume numerical approximation represents the depth averaged, 2D shallow-water equations described before. The water surface elevation, $\eta$, is defined at the center of each computational volume (nodes). Water depth, $H$, and velocity components, $U$ and $V$, are defined at the midpoint of volume faces (links). The finite volume structure provides a control volume representation that is inherently mass conservative.

The combination of a semi-implciit water surface elevation solution and a semi-Lagrangian representation of advection provides the advantages of a stable solution and of time steps that exceed the CFL criterion. In the semi-implicit process, $\eta$ in the momentum equations, and the velocity divergence in the continuity equation, are treated implicitly. The advective terms in the momentum equations, are discretized explicitly. See the cited literature for more details.

### The component

Import the needed libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm import trange

from landlab import RasterModelGrid
from landlab.components import RiverFlowDynamics
from landlab.io import esri_ascii
from landlab.plot.imshow import imshow_grid

## Information about the component

Using the class name as argument for the `help` function returns descriptions of the various methods and parameters.

In [ ]:
help(RiverFlowDynamics)

## Examples

-- --

### Example 1: Flow in a rectangular channel 6.0 m long

This first basic example illustrates water flowing through a rectangular channel 1.0 $m$ wide and 6.0 $m$ long. Our channel is made in concrete, so we choose a Manning's roughness coefficient equal to 0.012 $s/m^\frac{1}{3}$, and it has a slope of 0.01 $m/m$.

We specify some basic parameters such as the grid resolution, time step duration, number of time steps, and the domain dimensions by specifying the number of columns and rows. 

In [ ]:
# Basic parameters
mannings_n = 0.012  # Manning's roughness coefficient, [s/m^(1/3)]
channel_slope = 0.01  # Channel slope [m/m]

# Simulation parameters
n_timesteps = 1000  # Number of timesteps
dt = 0.1  # Timestep duration, [s]
nrows = 20  # Number of node rows
ncols = 60  # Number of node cols
dx = 0.1  # Node spacing in the x-direction, [m]
dy = 0.1  # Node spacing in the y-direction, [m]

Create the grid:

In [ ]:
# Create and set up the grid
grid = RasterModelGrid((nrows, ncols), xy_spacing=(dx, dy))

Create the elevation field and define the topography to represent our rectangular channel:

In [ ]:
# The grid represents a basic rectangular channel with slope equal to 0.01 m/m
te = grid.add_field(
    "topographic__elevation", 1.0 - channel_slope * grid.x_of_node, at="node"
)
te[grid.y_of_node > 1.5] = 2.5
te[grid.y_of_node < 0.5] = 2.5

We show a top view of the domain:

In [ ]:
# Showing the topography with improved visualization
plt.figure(figsize=(12, 6))
im = imshow_grid(grid, "topographic__elevation", cmap="terrain")
plt.colorbar(im, label="Elevation (m)")
plt.title("Channel Topography")
plt.show()

The channel is empty at the beginning of the simulation, so we create the fields for the water surface elevation, depth and velocity:

In [ ]:
# We establish the initial conditions, which represent an empty channel
h = grid.add_zeros("surface_water__depth", at="node")

# Water velocity is zero in everywhere since there is no water yet
vel = grid.add_zeros("surface_water__velocity", at="link")

# Calculating the initial water surface elevation from water depth and topographic elevation
wse = grid.add_field("surface_water__elevation", te, at="node")

Then, we specify the nodes at which water is entering into the domain, and also the associated links. These are going to be the entry boundary conditions for water depth and velocity. In this case, water flows from left to right at 0.5 $m$ depth, with a velocity of 0.45 $m/s$:

In [ ]:
# We set fixed boundary conditions, specifying the nodes and links in which the water is flowing into the grid
fixed_entry_nodes = np.array([300, 360, 420, 480, 540, 600, 660, 720, 780, 840, 900])
fixed_entry_links = grid.links_at_node[fixed_entry_nodes][:, 0]

# We set the fixed values in the entry nodes/links
entry_nodes_h_values = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
entry_links_vel_values = np.array(
    [0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45]
)

And now we show the boundary condition in the cross-section with improved visualization:

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(
    grid.y_of_node[fixed_entry_nodes], entry_nodes_h_values + te[fixed_entry_nodes],
    'b-', linewidth=2, label='Water Surface'
)
plt.plot(grid.y_of_node[grid.nodes_at_left_edge], te[grid.nodes_at_left_edge],
        'brown', linewidth=2, label='Channel Bottom')
plt.title("Channel Cross-section at Entry Boundary", fontsize=14)
plt.xlabel("Distance across channel [m]", fontsize=12)
plt.ylabel("Elevation [m]", fontsize=12)
plt.axis([0.25, 1.75, 0.75, 2.75])
plt.grid(True)
plt.legend()
plt.show()

We construct our component by passing the arguments we defined previously:

In [ ]:
# Finally, we run the model and let the water fill our channel
rfd = RiverFlowDynamics(
    grid,
    dt=dt,
    mannings_n=mannings_n,
    fixed_entry_nodes=fixed_entry_nodes,
    fixed_entry_links=fixed_entry_links,
    entry_nodes_h_values=entry_nodes_h_values,
    entry_links_vel_values=entry_links_vel_values,
)

And finally, we run the simulation for 1000 timesteps (100 seconds) with improved animation.

In [ ]:
# Set the animation frequency to n_timesteps if you
# don't want to plot the water depth
# display_animation_freq = n_timesteps
display_animation_freq = 5

for timestep in trange(n_timesteps):
    rfd.run_one_step()

    if timestep % display_animation_freq == 0:
        clear_output(wait=True)  # This will clear the previous image
        plt.figure(figsize=(12, 6))
        im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
        plt.colorbar(im, label="Water Depth (m)")
        plt.title(f"Water Depth at Time Step {timestep+1}")
        plt.show()

Exploring the water depth results at the latest time with enhanced visualization:

In [ ]:
plt.figure(figsize=(12, 6))
im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
plt.colorbar(im, label="Water Depth (m)")
plt.title(f"Final Water Depth after {n_timesteps} Time Steps")
plt.show()

And the water surface elevation with enhanced visualization:

In [ ]:
plt.figure(figsize=(12, 6))
im = imshow_grid(grid, "surface_water__elevation", cmap="terrain")
plt.colorbar(im, label="Water Surface Elevation (m)")
plt.title(f"Final Water Surface Elevation after {n_timesteps} Time Steps")
plt.show()

Let's also visualize a longitudinal profile of the water depth and water surface elevation along the channel centerline:

In [ ]:
# Get center row of the channel
center_row = nrows // 2
center_nodes = np.arange(center_row * ncols, (center_row + 1) * ncols, dtype=int)

# Plot water depth along centerline
plt.figure(figsize=(12, 6))
plt.plot(grid.x_of_node[center_nodes], grid.at_node['surface_water__depth'][center_nodes], 
         'b-', linewidth=2, label='Water Depth')
plt.grid(True)
plt.xlabel('Distance along channel (m)', fontsize=12)
plt.ylabel('Water Depth (m)', fontsize=12)
plt.title('Longitudinal Profile of Water Depth along Channel Centerline', fontsize=14)
plt.legend()
plt.show()

# Plot water surface elevation and topography along centerline
plt.figure(figsize=(12, 6))
plt.plot(grid.x_of_node[center_nodes], grid.at_node['surface_water__elevation'][center_nodes], 
         'b-', linewidth=2, label='Water Surface')
plt.plot(grid.x_of_node[center_nodes], grid.at_node['topographic__elevation'][center_nodes], 
         'brown', linewidth=2, label='Channel Bottom')
plt.grid(True)
plt.xlabel('Distance along channel (m)', fontsize=12)
plt.ylabel('Elevation (m)', fontsize=12)
plt.title('Longitudinal Profile of Water Surface and Channel Bottom', fontsize=14)
plt.legend()
plt.show()

## Interpretation of Results

The simulation shows the development of water flow in a rectangular channel with a constant inflow at the left boundary. Key observations from the results include:

1. **Water propagation**: The water enters the channel at the left boundary and gradually fills the entire length of the channel over time.

2. **Steady-state flow**: After sufficient time steps, the flow approaches a steady-state condition where the water depth stabilizes along the channel.

3. **Surface profile**: The water surface profile shows a gradual decrease in elevation in the downstream direction, consistent with the channel slope (0.01 m/m).

4. **Velocity distribution**: While not explicitly visualized, the flow velocity is influenced by the channel geometry and the friction coefficient (Manning's n = 0.012), typical for concrete channels.

This simulation demonstrates the capability of the RiverFlowDynamics component to model basic channel hydraulics, which is fundamental for understanding river systems, designing water infrastructure, and predicting flood behavior.

-- --
### And that's it! 

Nice work completing this tutorial. You know now how to use the `RiverFlowDynamics` Landlab component to run your own simulations :)

-- --



### Click here for more <a href="https://landlab.csdms.io/tutorials/">Landlab tutorials</a>